In [1]:
# %pip install pandas
# %pip install scikit-learn
# %pip install plotly
# %pip install seaborn
# %pip install numpy
# %pip install matplotlib
# %pip install nbformat
# %pip install opencv-python
# %pip install tensorflow
# %pip install wandb

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import cv2
import os
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import wandb
%env "WANDB_NOTEBOOK_NAME" "model_balanced.ipynb"

wandb.login()

from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: "WANDB_NOTEBOOK_NAME"="model_balanced.ipynb"


wandb: Currently logged in as: pedro-mariani (dspro2-group9). Use `wandb login --relogin` to force relogin


In [23]:
import tensorflow as tf
new_model = tf.keras.models.load_model("../../../model/240522_vgg16_model.keras")

In [24]:
new_model.save("../../../model/240522_vgg16_model.h5")

In [3]:
label_df = pd.read_csv(
    "../../../data/processed/sentiment/sentiment_label.csv", index_col=False
)
data_dir = "../../../data/processed/sentiment/img/"

label_df.head()

,image_name,sentiment
0,angry_0.jpg,negative
1,angry_1.jpg,negative
2,fear_2.jpg,negative
3,happy_3.jpg,positive
4,angry_4.jpg,negative


In [4]:
def load_data(dir_path, img_size):
    X = []
    y = []

    for index, row in label_df.iterrows():
        image_path = os.path.join(dir_path, row["image_name"])
        img = cv2.imread(image_path)
        img = (
            cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype("float32")
            / 255
        )
        X.append(img)
        y.append(row["sentiment"])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [5]:
IMG_SIZE = (48, 48)

X, y = load_data(data_dir, IMG_SIZE)

In [6]:
dummies = pd.get_dummies(label_df["sentiment"])
dummies.head()

,negative,neutral,positive
0,True,False,False
1,True,False,False
2,True,False,False
3,False,False,True
4,True,False,False


In [7]:
y = dummies.values
y

array([[ True, False, False],
       [ True, False, False],
       [ True, False, False],
       ...,
       [ True, False, False],
       [False, False,  True],
       [ True, False, False]])

In [8]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(91714, 48, 48, 3)
(11465, 48, 48, 3)
(11465, 48, 48, 3)
(91714, 3)
(11465, 3)
(11465, 3)


# Distribution of classes in each set


In [9]:
class_count_in_train_set = np.sum(y_train, axis=0)

print("Class distribution:")
print(class_count_in_train_set)

Class distribution:
[28822 31268 31624]


In [10]:
labels = ["positive", "neutral", "negative"]
fig = px.bar(x=labels, y=class_count_in_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set",
    xaxis_title="Sentiments",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [11]:
class_count_in_validation_set = np.sum(y_valid, axis=0)

print("Class distribution:")
print(class_count_in_validation_set)

Class distribution:
[3610 3936 3919]


In [12]:
labels = ["positive", "neutral", "negative"]
fig = px.bar(x=labels, y=class_count_in_validation_set, color=labels)

fig.update_layout(
    title="Distribution of each label in validation set",
    xaxis_title="Sentiments",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [13]:
class_count_in_test_set = np.sum(y_test, axis=0)

print("Class distribution:")
print(class_count_in_test_set)

Class distribution:
[3579 3903 3983]


In [14]:
labels = ["positive", "neutral", "negative"]
fig = px.bar(x=labels, y=class_count_in_test_set, color=labels)

fig.update_layout(
    title="Distribution of each label in test set",
    xaxis_title="Sentiments",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Data augmentation to balance the dataset


In [15]:
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     brightness_range=(0.8, 1.2),
#     zoom_range=0.1,
# )

# augmented_images = datagen.flow(X_train, y_train, batch_size=len(X_train))

# x_train_augmented = np.concatenate(
#     [
#         X_train,
#         augmented_images[0][0],
#     ]
# )
# y_train_augmented = np.concatenate(
#     [
#         y_train,
#         augmented_images[0][1],
#     ]
# )

# Model training


In [16]:
IMG_SIZE = (48, 48)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False

base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [17]:
NUM_CLASSES = 3

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [18]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    name="vgg16_three_sentiments_epoch50_batch32",
    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "Adam(learning_rate=3e-4)",
        "loss": "categorical_crossentropy",
        "metric": ["categorical_accuracy"],
        "epoch": 50,
        "batch_size": 32,
        "architecture": "vgg16",
    },
)

In [19]:
config = wandb.config


model.compile(
    loss=config["loss"],
    optimizer=Adam(learning_rate=3e-4),
    metrics=config["metric"],
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=config["epoch"],
    batch_size=config["batch_size"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)


wandb.finish()

NameError: name 'Adam' is not defined

In [ ]:
y_pred = model.predict(X_valid)
true_class = tf.argmax(y_valid, 1)
predicted_class = tf.argmax(y_pred, 1)

In [ ]:
cm = confusion_matrix(true_class, predicted_class)

ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax=ax)

# labels, title and ticks
ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")
ax.set_title("Confusion Matrix")
ax.xaxis.set_ticklabels(
    ["positive", "neutral", "negative"]
)
ax.yaxis.set_ticklabels(
    ["positive", "neutral", "negative"]
)